In [49]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm

# 参数设定
city = 'shanghai'
time = [2018, 9, 21]

GRID_SIZE = 50  # meter
target_dt = datetime.datetime(time[0], time[1], time[2], 8, 0)
target_hours = list(range(8, 21))

In [50]:
# 区域边界
if city == 'beijing':
    lon_l, lon_r, lat_b, lat_u = 115.43, 117.52, 39.44, 41.05
if city == 'shanghai':
    lon_l, lon_r, lat_b, lat_u = 120.836619, 122.165824, 30.631414, 31.7 #31.883925
if city == 'shenzhen':
    lon_l, lon_r, lat_b, lat_u = 113.70, 114.70, 22.45, 22.85
GRID_SIZE = 50  # Meter

# 经纬度步长计算
earth_radius = 6378137.0
pi = 3.1415926535897932384626
meter_per_degree = earth_radius * pi / 180.0
lat_step = GRID_SIZE * (1.0 / meter_per_degree)
ratio = np.cos((lat_b + lat_u) * np.pi / 360)
lon_step = lat_step / ratio
lat_split = int(np.ceil((lat_u - lat_b) / lat_step))
lon_split = int(np.ceil((lon_r - lon_l) / lon_step))

# 起始时间（小时偏移起点）
t0 = datetime.datetime.strptime('20180101 00:00:00', "%Y%m%d %H:%M:%S")

# 读取函数
def read_from_text(file):
    for line in file:
        yield line.strip('\r\n').split('\t')

# 映射回真实经纬度（用于可视化时反查）
def index_to_lon(x_idx):
    return lon_l + x_idx * lon_step

def index_to_lat(y_idx):
    return lat_b + y_idx * lat_step

In [51]:
if time[2] < 10:
    TMP = np.load(f'./tmp_sim/tmp_sim_{city}_2018090{str(time[2])}_cube.npy') # tmp_level_matrix
else:
    TMP = np.load(f'./tmp_sim/tmp_sim_{city}_201809{str(time[2])}_cube.npy') # tmp_level_matrix

In [52]:
t0 = datetime.datetime(2018, 1, 1, 0, 0, 0)

target_start = int((target_dt - t0).total_seconds() // 3600)
target_end = target_start + 12  # 到 20:00
target_hours = list(range(target_start, target_end + 1))  # 13个小时点

all_trace_heat_level = []
all_trace_step = []

all_steps = []
all_tmp_start = []
all_tmp_end = []
all_weeknum = []
all_clock = []

with open(f'./record/record_{city}.txt', 'r') as file:
    for uid, trace in tqdm(read_from_text(file), desc="Processing traces"):
        trace = trace.split('|')

        t_list = [int(trace[i]) for i in range(0, len(trace), 3)]
        x_list = [float(trace[i]) for i in range(1, len(trace), 3)]
        y_list = [float(trace[i]) for i in range(2, len(trace), 3)]

        if not (min(t_list) <= target_start and max(t_list) >= target_end):
            continue  # 不满足插值条件

        if not (max(x_list) <= lon_split and max(y_list) <= lat_split):
            continue  # 不满足插值条件

        # 提取有效点
        valid_t = np.array(t_list)
        valid_x = np.array(x_list)
        valid_y = np.array(y_list)

        # 插值坐标
        interp_x = np.round(np.interp(target_hours, valid_t, valid_x))
        interp_y = np.round(np.interp(target_hours, valid_t, valid_y))

        new_x_list, new_y_list = interp_x.tolist(), interp_y.tolist()
        # trace_heat_level = [TLM[int(new_y_list[i]), int(new_x_list[i])] for i in range(len(new_x_list))]
        # all_trace_heat_level.append(trace_heat_level)

        if max(new_x_list) < lon_split and max(new_y_list) < lat_split:
            for i in range(11):
                all_steps.append(np.sqrt(((new_x_list[i+1]-new_x_list[i])/lon_step)**2 + ((new_y_list[i+1]-new_y_list[i])/lat_step)**2))
                all_tmp_start.append(TMP[i, int(new_y_list[i]), int(new_x_list[i])])
                all_tmp_end.append(TMP[i+1, int(new_y_list[i+1]), int(new_x_list[i+1])])
                all_weeknum.append(5)
                all_clock.append(i+8)

        trace_step = [np.sqrt(((new_x_list[i+1]-new_x_list[i])/lon_step)**2 + ((new_y_list[i+1]-new_y_list[i])/lat_step)**2) for i in range(len(new_x_list)-1)]
        all_trace_step.append(trace_step)

Processing traces: 450695it [03:40, 2047.32it/s]


In [53]:
all_steps, all_tmp_start, all_tmp_end, all_weeknum, all_clock = np.array(all_steps), np.array(all_tmp_start), np.array(all_tmp_end), np.array(all_weeknum), np.array(all_clock)

np.savez(f'./stata/stata_{city}_{target_dt.strftime("%Y%m%d_%H%M")}.npz', steps=all_steps, tmp_start=all_tmp_start, tmp_end=all_tmp_end, weeknum=all_weeknum, clock=all_clock)